In [234]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *

pd.set_option('display.max_columns', None)

In [235]:
DATA_PATH_M = 'C:/Users/FLUXNATURE/Desktop/New Kaggle world/NCAAM/'

for filename in os.listdir(DATA_PATH_M):
    print(filename)

Cities.csv
Conferences.csv
MConferenceTourneyGames.csv
MGameCities.csv
MMasseyOrdinals.csv
MNCAATourneyCompactResults.csv
MNCAATourneyDetailedResults.csv
MNCAATourneySeedRoundSlots.csv
MNCAATourneySeeds.csv
MNCAATourneySlots.csv
MRegularSeasonCompactResults.csv
MRegularSeasonDetailedResults.csv
MSampleSubmissionStage1.csv
MSeasons.csv
MSecondaryTourneyCompactResults.csv
MSecondaryTourneyTeams.csv
MTeamCoaches.csv
MTeamConferences.csv
MTeams.csv
MTeamSpellings.csv


DATA PREPARATION AND PROCESSING

Data: WNCAATourneySeeds.csv

"This file identifies the seeds for all teams in each NCAA® tournament, for all seasons of historical data. Thus, there are exactly 64 rows for each year, since there are no play-in teams in the women's tournament. We will not know the seeds of the respective tournament teams, or even exactly which 64 teams it will be, until Selection Monday on March 16, 2020 (DayNum=133).

Season - the year that the tournament was played in Seed - this is a 3-character identifier of the seed, where the first character is either W, X, Y, or Z (identifying the region the team was in) and the next two digits (either 01, 02, ..., 15, or 16) tell you the seed within the region. For example, the first record in the file is seed W01, which means we are looking at the #1 seed in the W region (which we can see from the "WSeasons.csv" file was the East region). TeamID - this identifies the id number of the team, as specified in the WTeams.csv file"

In [236]:
 df_seeds = pd.read_csv(DATA_PATH_M + "MNCAATourneySeeds.csv")
 df_seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


SEASON'S RESULTS

Data: WRegularSeasonCompactResults.csv

This file identifies the game-by-game results for many seasons of historical data, starting with the 1998 season. For each season, the file includes all games played from DayNum 0 through 132. It is important to realize that the "Regular Season" games are simply defined to be all games played on DayNum=132 or earlier (DayNum=133 is Selection Monday). Thus a game played before Selection Monday will show up here whether it was a pre-season tournament, a non-conference game, a regular conference game, a conference tournament game, or whatever.

Season - this is the year of the associated entry in WSeasons.csv (the year in which the final tournament occurs). For example, during the 2016 season, there were regular season games played between November 2015 and March 2016, and all of those games will show up with a Season of 2016.

DayNum - this integer always ranges from 0 to 132, and tells you what day the game was played on. It represents an offset from the "DayZero" date in the "WSeasons.csv" file. For example, the first game in the file was DayNum=18. Combined with the fact from the "WSeasons.csv" file that day zero was 10/27/1997 that year, this means the first game was played 18 days later, or 11/14/1997. There are no teams that ever played more than one game on a given date, so you can use this fact if you need a unique key (combining Season and DayNum and WTeamID).

WTeamID - this identifies the id number of the team that won the game, as listed in the "WTeams.csv" file. No matter whether the game was won by the home team or visiting team, or if it was a neutral-site game, the "WTeamID" always identifies the winning team.

WScore - this identifies the number of points scored by the winning team.

LTeamID - this identifies the id number of the team that lost the game.

LScore - this identifies the number of points scored by the losing team. Thus you can be confident that WScore will be greater than LScore for all games listed.

NumOT - this indicates the number of overtime periods in the game, an integer 0 or higher.

WLoc - this identifies the "location" of the winning team. If the winning team was the home team, this value will be "H". If the winning team was the visiting team, this value will be "A". If it was played on a neutral court, then this value will be "N".



In [237]:
#dropping NumOT and WLoc from the data 
df_season_results = pd.read_csv(DATA_PATH_M + "MRegularSeasonCompactResults.csv")
df_season_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

In [238]:
df_season_results['ScoreGap'] = df_season_results['WScore'] - df_season_results['LScore']

In [239]:
df_season_results.head(4)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,ScoreGap
0,1985,20,1228,81,1328,64,17
1,1985,25,1106,77,1354,70,7
2,1985,25,1112,63,1223,56,7
3,1985,25,1165,70,1432,54,16


Features
For each team at each season, I compute :

Number of wins

Number of losses

Average score gap of wins

Average score gap of losses

And use the following features :

Win Ratio
Average score gap

In [240]:
num_win = df_season_results.groupby(['Season', 'WTeamID']).count()
num_win = num_win.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})

In [241]:
num_loss = df_season_results.groupby(['Season', 'LTeamID']).count()
num_loss = num_loss.reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "NumLosses", "LTeamID": "TeamID"})

In [242]:
gap_win = df_season_results.groupby(['Season', 'WTeamID']).mean().reset_index()
gap_win = gap_win[['Season', 'WTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapWins", "WTeamID": "TeamID"})

In [243]:
gap_loss = df_season_results.groupby(['Season', 'LTeamID']).mean().reset_index()
gap_loss = gap_loss[['Season', 'LTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapLosses", "LTeamID": "TeamID"})

MERGE COMPUTATIONS 

In [244]:
df_features_season_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_features_season_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

In [245]:
df_features_season = pd.concat([df_features_season_w, df_features_season_l], 0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

In [246]:
df_features_season = df_features_season.merge(num_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(num_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_loss, on=['Season', 'TeamID'], how='left')

In [247]:
df_features_season.fillna(0, inplace=True) 

FEATURES COMPUTATION

In [248]:
df_features_season['WinRatio'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])
df_features_season['GapAvg'] = (
    (df_features_season['NumWins'] * df_features_season['GapWins'] - 
    df_features_season['NumLosses'] * df_features_season['GapLosses'])
    / (df_features_season['NumWins'] + df_features_season['NumLosses'])
)

In [249]:
df_features_season.drop(['NumWins', 'NumLosses', 'GapWins', 'GapLosses'], axis=1, inplace=True)

TOURNEY

Data: WNCAATourneyCompactResults.csv

This file identifies the game-by-game NCAA® tournament results for all seasons of historical data. The data is formatted exactly like the WRegularSeasonCompactResults data. Each season you will see 63 games listed, since there are no women's play-in games.

Although the scheduling of the men's tournament rounds has been consistent for many years, there has been more variety in the scheduling of the women's rounds. There have been four different schedules over the course of the past 20+ years for the women's tournament, as follows:

In [250]:
df_tourney_results = pd.read_csv(DATA_PATH_M + "MNCAATourneyCompactResults.csv")
df_tourney_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

The DayNum features can be improved by replacing it by the corresponding round.

In [251]:
def get_round(day):
    round_dic = {134: 0, 135: 0, 136: 1, 137: 1, 138: 2, 139: 2, 143: 3, 144: 3, 145: 4, 146: 4, 152: 5, 154: 6}
    try:
        return round_dic[day]
    except:
        print(f'Unknow day : {day}')
        return 0


In [252]:
df_tourney_results['Round'] = df_tourney_results['DayNum'].apply(get_round)

In [253]:
df_tourney_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round
0,1985,136,1116,63,1234,54,1
1,1985,136,1120,59,1345,58,1
2,1985,136,1207,68,1250,43,1
3,1985,136,1229,58,1425,55,1
4,1985,136,1242,49,1325,38,1


In [254]:
df_tourney_results.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round
2246,2019,146,1120,77,1246,71,4
2247,2019,146,1277,68,1181,67,4
2248,2019,152,1403,61,1277,51,5
2249,2019,152,1438,63,1120,62,5
2250,2019,154,1438,85,1403,77,6


RATINGS 

Massey Ordinals
This file lists out rankings (e.g. #1, #2, #3, ..., #N) of teams going back to the 2002-2003 season, under a large number of different ranking system methodologies.

Season - this is the year of the associated entry in MSeasons.csv (the year in which the final tournament occurs)

RankingDayNum - First day that it is appropriate to use the rankings for predicting games. Use 133 for the tournament.

SystemName - this is the (usually) 3-letter abbreviation for each distinct ranking system.

TeamID - this is the ID of the team being ranked, as described in MTeams.csv.

OrdinalRank - this is the overall ranking of the team in the underlying system. Most systems from recent seasons provide a 

complete ranking from #1 through #351, but sometimes there are ties and sometimes only a smaller set of rankings is provided, as with the AP's top 25. This year and last year they will typically go up to #353 because two new teams were added to Division I last year.


In [255]:
 df_massey = pd.read_csv(DATA_PATH_M + "MMasseyOrdinals.csv")
 df_massey = df_massey[df_massey['RankingDayNum'] == 133].drop('RankingDayNum', axis=1).reset_index(drop=True) # use first day of the tournament


In [256]:
df_massey.tail()

,Season,SystemName,TeamID,OrdinalRank
291428,2019,ZAM,1462,70
291429,2019,ZAM,1463,87
291430,2019,ZAM,1464,242
291431,2019,ZAM,1465,198
291432,2019,ZAM,1466,290


Processing
I keep only systems that are common to all the years.

In [257]:
systems = []
for year in range(2003, 2019):
     r = df_massey[df_massey['Season'] == year]
     systems.append(r['SystemName'].unique())
    
all_systems = list(set(list(np.concatenate(systems))))

In [258]:
common_systems = []  
for system in all_systems:
     common = True
     for system_years in systems:
         if system not in system_years:
             common = False
     if common:
         common_systems.append(system)
        
common_systems


['WOL', 'POM', 'MOR', 'SAG', 'DOL', 'RTH', 'USA', 'WLK', 'COL', 'AP', 'RPI']

In [259]:
f_massey = df_massey[df_massey['SystemName'].isin(common_systems)].reset_index(drop=True)

FEATURE ENGINEERING 

TRAIN DATA

In [260]:
df = df_tourney_results.copy()
df = df[df['Season'] >= 2003].reset_index(drop=True)

df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round
0,2003,134,1421,92,1411,84,0
1,2003,136,1112,80,1436,51,1
2,2003,136,1113,84,1272,71,1
3,2003,136,1141,79,1166,73,1
4,2003,136,1143,76,1301,74,1


In [261]:
df.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round
1110,2019,146,1120,77,1246,71,4
1111,2019,146,1277,68,1181,67,4
1112,2019,152,1403,61,1277,51,5
1113,2019,152,1438,63,1120,62,5
1114,2019,154,1438,85,1403,77,6


Each row corresponds to a match between WTeamID and LTeamID, which was won by WTeamID.

I only keep matches after 2003 since I don't have the ratings for the older ones.

I start by aggregating features coresponding to each tem.

Seeds

SeedW is the seed of the winning team

SeedL is the seed of the losing team


In [262]:
df = pd.merge(
    df, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'WTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedW'})

In [263]:
df = pd.merge(
    df, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'LTeamID'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedL'})

In [264]:
def treat_seed(seed):
    return int(re.sub("[^0-9]", "", seed))

In [265]:
df['SeedW'] = df['SeedW'].apply(treat_seed)
df['SeedL'] = df['SeedL'].apply(treat_seed)

In [266]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round,SeedW,SeedL
0,2003,134,1421,92,1411,84,0,16,16
1,2003,136,1112,80,1436,51,1,1,16
2,2003,136,1113,84,1272,71,1,10,7
3,2003,136,1141,79,1166,73,1,11,6
4,2003,136,1143,76,1301,74,1,8,9


Season Stats

WinRatioW is the win ratio of the winning team during the season

WinRatioL is the win ratio of the losing team during the season

In [267]:
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsW',
    'NumLosses': 'NumLossesW',
    'GapWins': 'GapWinsW',
    'GapLosses': 'GapLossesW',
    'WinRatio': 'WinRatioW',
    'GapAvg': 'GapAvgW',
}).drop(columns='TeamID', axis=1)

In [268]:
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsL',
    'NumLosses': 'NumLossesL',
    'GapWins': 'GapWinsL',
    'GapLosses': 'GapLossesL',
    'WinRatio': 'WinRatioL',
    'GapAvg': 'GapAvgL',
}).drop(columns='TeamID', axis=1)

In [269]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round,SeedW,SeedL,WinRatioW,GapAvgW,WinRatioL,GapAvgL
0,2003,134,1421,92,1411,84,0,16,16,0.448276,-7.241379,0.600000,1.966667
1,2003,136,1112,80,1436,51,1,1,16,0.892857,14.964286,0.655172,4.655172
2,2003,136,1113,84,1272,71,1,10,7,0.620690,6.793103,0.793103,8.689655
3,2003,136,1141,79,1166,73,1,11,6,0.793103,6.103448,0.878788,14.909091
4,2003,136,1143,76,1301,74,1,8,9,0.724138,4.724138,0.600000,4.400000


Ratings

OrdinalRankW is the average Massey Ranking of the winning team

OrdinalRankL is the average Massey Ranking of the losing team

In [270]:
avg_ranking = df_massey.groupby(['Season', 'TeamID']).mean().reset_index()

In [271]:
df = pd.merge(
     df,
     avg_ranking,
     how='left',
     left_on=['Season', 'WTeamID'],
     right_on=['Season', 'TeamID']
 ).drop('TeamID', axis=1).rename(columns={'OrdinalRank': 'OrdinalRankW'})

In [272]:
df = pd.merge(
     df, 
     avg_ranking, 
     how='left', 
     left_on=['Season', 'LTeamID'], 
     right_on=['Season', 'TeamID']
 ).drop('TeamID', axis=1).rename(columns={'OrdinalRank': 'OrdinalRankL'})

In [273]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,Round,SeedW,SeedL,WinRatioW,GapAvgW,WinRatioL,GapAvgL,OrdinalRankW,OrdinalRankL
0,2003,134,1421,92,1411,84,0,16,16,0.448276,-7.241379,0.600000,1.966667,240.343750,239.281250
1,2003,136,1112,80,1436,51,1,1,16,0.892857,14.964286,0.655172,4.655172,2.676471,153.125000
2,2003,136,1113,84,1272,71,1,10,7,0.620690,6.793103,0.793103,8.689655,36.000000,21.705882
3,2003,136,1141,79,1166,73,1,11,6,0.793103,6.103448,0.878788,14.909091,45.687500,20.735294
4,2003,136,1143,76,1301,74,1,8,9,0.724138,4.724138,0.600000,4.400000,36.406250,50.312500


Add symetrical

Right now our data only consists of won matches

We duplicate our data, get rid of the winner loser

In [274]:
def add_loosing_matches(win_df):
    win_rename = {
        "WTeamID": "TeamIdA", 
        "WScore" : "ScoreA", 
        "LTeamID" : "TeamIdB",
        "LScore": "ScoreB",
        "SeedW": "SeedA", 
        "SeedL": "SeedB",
        'WinRatioW' : 'WinRatioA',
        'WinRatioL' : 'WinRatioB',
        'GapAvgW' : 'GapAvgA',
        'GapAvgL' : 'GapAvgB',
        "OrdinalRankW": "OrdinalRankA",
         "OrdinalRankL": "OrdinalRankB",
     }
    
    lose_rename = {
        "WTeamID": "TeamIdB", 
        "WScore" : "ScoreB", 
        "LTeamID" : "TeamIdA",
        "LScore": "ScoreA",
        "SeedW": "SeedB", 
        "SeedL": "SeedA",
        'GapAvgW' : 'GapAvgB',
        'GapAvgL' : 'GapAvgA',
        'WinRatioW' : 'WinRatioB',
        'WinRatioL' : 'WinRatioA',
         "OrdinalRankW": "OrdinalRankB",
         "OrdinalRankL": "OrdinalRankA",
    }
    
    win_df = win_df.copy()
    lose_df = win_df.copy()
    
    win_df = win_df.rename(columns=win_rename)
    lose_df = lose_df.rename(columns=lose_rename)
    
    return pd.concat([win_df, lose_df], 0, sort=False)

In [275]:
df = add_loosing_matches(df)

Differences

We compute the difference between the team for each feature.

This helps further assessing how better (or worse) team A is from team B


In [276]:
df['SeedDiff'] = df['SeedA'] - df['SeedB']
df['WinRatioDiff'] = df['WinRatioA'] - df['WinRatioB']
df['GapAvgDiff'] = df['GapAvgA'] - df['GapAvgB']
df['OrdinalRankDiff'] = df['OrdinalRankA'] - df['OrdinalRankB']

In [277]:
df.head()

,Season,DayNum,TeamIdA,ScoreA,TeamIdB,ScoreB,Round,SeedA,SeedB,WinRatioA,GapAvgA,WinRatioB,GapAvgB,OrdinalRankA,OrdinalRankB,SeedDiff,WinRatioDiff,GapAvgDiff,OrdinalRankDiff
0,2003,134,1421,92,1411,84,0,16,16,0.448276,-7.241379,0.600000,1.966667,240.343750,239.281250,0,-0.151724,-9.208046,1.062500
1,2003,136,1112,80,1436,51,1,1,16,0.892857,14.964286,0.655172,4.655172,2.676471,153.125000,-15,0.237685,10.309113,-150.448529
2,2003,136,1113,84,1272,71,1,10,7,0.620690,6.793103,0.793103,8.689655,36.000000,21.705882,3,-0.172414,-1.896552,14.294118
3,2003,136,1141,79,1166,73,1,11,6,0.793103,6.103448,0.878788,14.909091,45.687500,20.735294,5,-0.085684,-8.805643,24.952206
4,2003,136,1143,76,1301,74,1,8,9,0.724138,4.724138,0.600000,4.400000,36.406250,50.312500,-1,0.124138,0.324138,-13.906250


TEST  DATA 

PREPARING

In [278]:
df_test = pd.read_csv(DATA_PATH_M + "MSampleSubmissionStage1.csv")

In [279]:
df_test['Season'] = df_test['ID'].apply(lambda x: int(x.split('_')[0]))
df_test['TeamIdA'] = df_test['ID'].apply(lambda x: int(x.split('_')[1]))
df_test['TeamIdB'] = df_test['ID'].apply(lambda x: int(x.split('_')[2]))

In [280]:
df_test.head()

,ID,Pred,Season,TeamIdA,TeamIdB
0,2015_1107_1112,0.5,2015,1107,1112
1,2015_1107_1116,0.5,2015,1107,1116
2,2015_1107_1124,0.5,2015,1107,1124
3,2015_1107_1125,0.5,2015,1107,1125
4,2015_1107_1129,0.5,2015,1107,1129


SEEDS

In [281]:
df_test = pd.merge(
    df_test,
    df_seeds,
    how='left',
    left_on=['Season', 'TeamIdA'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedA'})

In [282]:
df_test = pd.merge(
    df_test, 
    df_seeds, 
    how='left', 
    left_on=['Season', 'TeamIdB'], 
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedB'})

In [283]:
df_test['SeedA'] = df_test['SeedA'].apply(treat_seed)
df_test['SeedB'] = df_test['SeedB'].apply(treat_seed)

SEASON'S STATS

In [284]:
df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIdA'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsA',
    'NumLosses': 'NumLossesA',
    'GapWins': 'GapWinsA',
    'GapLosses': 'GapLossesA',
    'WinRatio': 'WinRatioA',
    'GapAvg': 'GapAvgA',
}).drop(columns='TeamID', axis=1)

In [285]:
df_test = pd.merge(
    df_test,
    df_features_season,
    how='left',
    left_on=['Season', 'TeamIdB'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'NumWins': 'NumWinsB',
    'NumLosses': 'NumLossesB',
    'GapWins': 'GapWinsB',
    'GapLosses': 'GapLossesB',
    'WinRatio': 'WinRatioB',
    'GapAvg': 'GapAvgB',
}).drop(columns='TeamID', axis=1)

RATINGS 

In [286]:
df_test = pd.merge(
     df_test,
     avg_ranking,
     how='left',
     left_on=['Season', 'TeamIdA'],
     right_on=['Season', 'TeamID']
 ).drop('TeamID', axis=1).rename(columns={'OrdinalRank': 'OrdinalRankA'})

In [287]:
df_test = pd.merge(
     df_test,
     avg_ranking,
     how='left',
     left_on=['Season', 'TeamIdB'],
     right_on=['Season', 'TeamID']
 ).drop('TeamID', axis=1).rename(columns={'OrdinalRank': 'OrdinalRankB'})

DIFFERENCES

In [288]:
df_test['SeedDiff'] = df_test['SeedA'] - df_test['SeedB']
df_test['WinRatioDiff'] = df_test['WinRatioA'] - df_test['WinRatioB']
df_test['GapAvgDiff'] = df_test['GapAvgA'] - df_test['GapAvgB']
df_test['OrdinalRankDiff'] = df_test['OrdinalRankA'] - df_test['OrdinalRankB']

In [289]:
df_test.head()

,ID,Pred,Season,TeamIdA,TeamIdB,SeedA,SeedB,WinRatioA,GapAvgA,WinRatioB,GapAvgB,OrdinalRankA,OrdinalRankB,SeedDiff,WinRatioDiff,GapAvgDiff,OrdinalRankDiff
0,2015_1107_1112,0.5,2015,1107,1112,14,2,0.75,5.28125,0.911765,17.823529,118.87931,4.209677,12,-0.161765,-12.542279,114.669633
1,2015_1107_1116,0.5,2015,1107,1116,14,5,0.75,5.28125,0.764706,7.882353,118.87931,22.306452,9,-0.014706,-2.601103,96.572859
2,2015_1107_1124,0.5,2015,1107,1124,14,3,0.75,5.28125,0.718750,8.812500,118.87931,14.177419,11,0.031250,-3.531250,104.701891
3,2015_1107_1125,0.5,2015,1107,1125,14,15,0.75,5.28125,0.677419,3.612903,118.87931,129.724138,-1,0.072581,1.668347,-10.844828
4,2015_1107_1129,0.5,2015,1107,1129,14,11,0.75,5.28125,0.741935,8.935484,118.87931,46.700000,3,0.008065,-3.654234,72.179310


TARGETS

In [290]:
df['ScoreDiff'] = df['ScoreA'] - df['ScoreB']
df['WinA'] = (df['ScoreDiff'] > 0).astype(int)

MODELLING

In [291]:
features = [
    'SeedA',
    'SeedB',
    'WinRatioA',
    'GapAvgA',
    'WinRatioB',
    'GapAvgB',
    'OrdinalRankA',
    'OrdinalRankB',
    'SeedDiff',
    'WinRatioDiff',
    'GapAvgDiff',
    'OrdinalRankDiff',
]

In [292]:
def rescale(features, df_train, df_val, df_test=None):
    min_ = df_train[features].min()
    max_ = df_train[features].max()
    
    df_train[features] = (df_train[features] - min_) / (max_ - min_)
    df_val[features] = (df_val[features] - min_) / (max_ - min_)
    
    if df_test is not None:
        df_test[features] = (df_test[features] - min_) / (max_ - min_)
        
    return df_train, df_val, df_test

Cross Validation

Validate on season n, for n in the 10 last seasons.

Train on earlier seasons

Pipeline support classification (predict the team that wins) and regression (predict the score gap)

In [293]:
def kfold_reg(df, df_test_=None, plot=False, verbose=0, mode="reg"):
    seasons = df['Season'].unique()
    cvs = []
    pred_tests = []
    target = "ScoreDiff" if mode == "reg" else "WinA"
    
    for season in seasons[10:]:
        if verbose:
            print(f'\nValidating on season {season}')
        
        df_train = df[df['Season'] < season].reset_index(drop=True).copy()
        df_val = df[df['Season'] == season].reset_index(drop=True).copy()
        df_test = df_test_.copy()
        
        df_train, df_val, df_test = rescale(features, df_train, df_val, df_test)
        
        if mode == "reg":
            model = ElasticNet(alpha=1, l1_ratio=0.5)
        else:
            model = LogisticRegression(C=10)
            
        model.fit(df_train[features], df_train[target])
        
        if mode == "reg":
            pred = model.predict(df_val[features])
            pred = (pred - pred.min()) / (pred.max() - pred.min())
        else:
            pred = model.predict_proba(df_val[features])[:, 1]
        
        if df_test is not None:
            if mode == "reg":
                pred_test = model.predict(df_test[features])
                pred_test = (pred_test - pred_test.min()) / (pred_test.max() - pred_test.min())
            else:
                pred_test = model.predict_proba(df_test[features])[:, 1]
                
            pred_tests.append(pred_test)
            
        if plot:
            plt.figure(figsize=(15, 6))
            plt.subplot(1, 2, 1)
            plt.scatter(pred, df_val['ScoreDiff'].values, s=5)
            plt.grid(True)
            plt.subplot(1, 2, 2)
            sns.histplot(pred)
            plt.show()
        
        loss = log_loss(df_val['WinA'].values, pred)
        cvs.append(loss)

        if verbose:
            print(f'\t -> Scored {loss:.3f}')
        
    print(f'\n Local CV is {np.mean(cvs):.3f}')
    
    return pred_tests

In [294]:
pred_tests = kfold_reg(df, df_test, plot=False, verbose=1, mode="cls")


Validating on season 2013
	 -> Scored 0.610

Validating on season 2014
	 -> Scored 0.585

Validating on season 2015
	 -> Scored 0.522

Validating on season 2016
	 -> Scored 0.565

Validating on season 2017
	 -> Scored 0.521

Validating on season 2018
	 -> Scored 0.610

Validating on season 2019
	 -> Scored 0.493

 Local CV is 0.558


Submission

Note that this pipeline is leaky during the first stage of the competition : the LB will be underestimated since the last 4

models were trained

In [295]:
pred_test = np.mean(pred_tests, 0)

In [296]:
sub = df_test[['ID', 'Pred']].copy()
sub['Pred'] = pred_test
sub.to_csv('submission_Ismail_AbooNaziha.csv', index=False)

In [297]:
sub.head()

,ID,Pred
0,2015_1107_1112,0.045023
1,2015_1107_1116,0.122837
2,2015_1107_1124,0.088381
3,2015_1107_1125,0.563249
4,2015_1107_1129,0.203074
